In [1]:
from inspect import signature

In [3]:
def test_func(x, y, z=1):
    return x + y + z

sig = signature(test_func)
print(sig)
help(sig.bind)

(x, y, z=1)
Help on method bind in module inspect:

bind(*args, **kwargs) method of inspect.Signature instance
    Get a BoundArguments object, that maps the passed `args`
    and `kwargs` to the function's signature.  Raises `TypeError`
    if the passed arguments can not be bound.



In [10]:

bound_args = sig.bind(1, 2)
print(bound_args)  # Outputs: <BoundArguments (x=1, y=2, z=10)>
print(bound_args.arguments)
print(bound_args.args)
print(bound_args.kwargs)
bound_args = sig.bind(x=1, y=2)
print(bound_args)  # Outputs: <BoundArguments (x=1, y=2, z=10)>

<BoundArguments (x=1, y=2)>
{'x': 1, 'y': 2}
(1, 2)
{}
<BoundArguments (x=1, y=2)>


In [6]:
result = test_func(*bound_args.args)
print(result)

4


In [12]:

partial_bound_args = sig.bind_partial(1)
print(partial_bound_args)  # Outputs: <BoundArguments (x=1)>

<BoundArguments (x=1)>


In [13]:
partial_bound_args.arguments['y'] = 2
if 'z' not in partial_bound_args.arguments:
    partial_bound_args.arguments['z'] = 20  # Update or provide more arguments later
args, kwargs = partial_bound_args.args, partial_bound_args.kwargs
result = test_func(*args, **kwargs)  # This effectively calls test_func(1, 2, 20)
print(result)  # Outputs: 23


23


# Dynamic Function Calls

In dynamic systems, especially where functions or methods may be invoked based on conditions or configurations that change at runtime, pre-validating arguments using bind() ensures that the function will not fail due to missing or unexpected arguments when it is finally called.

In [9]:

from inspect import signature

def plugin_a(name, data):
    print(f"Plugin A processing {name} with {data}")

def plugin_b(name, data, flag):
    print(f"Plugin B processing {name} with {data} and flag={flag}")

plugin_registry = {
    'a': plugin_a,
    'b': plugin_b
}

def call_plugin(plugin_name, **kwargs):
    plugin = plugin_registry[plugin_name]
    sig = signature(plugin)
    try:
        bound_args = sig.bind(**kwargs)
        plugin(**bound_args.arguments)
    except TypeError as e:
        print(f"Failed to call plugin {plugin_name}: {e}")

# Correct call
call_plugin('a', name='test', data='some data')

# Call with missing arguments
call_plugin('b', name='test')

Plugin A processing test with some data

Failed to call plugin b: missing a required argument: 'data'


# Validation of Arguments

The direct use of bind() in a testing or debugging environment ensures that functions are only called if they can be called correctly, thus preventing runtime errors and improving the stability of the application.

In [10]:

def process_data(x, y, option=False):
    if option:
        return x * y
    else:
        return x + y

def validate_args(func, *args, **kwargs):
    sig = signature(func)
    try:
        sig.bind(*args, **kwargs)
        print("Arguments are valid.")
    except TypeError as e:
        print(f"Argument validation failed: {e}")

validate_args(process_data, 1, 2)
validate_args(process_data, 1)  # Will fail


Arguments are valid.

Argument validation failed: missing a required argument: 'y'


# Partial Configuration# Partial Configuration

In systems where configuration data may not be available all at once, partial_bind() allows for a staged setup where arguments are validated and stored as they become available, preventing premature function execution.

In [11]:

def configure_system(operation, value, mode=None):
    print(f"Configuring system with operation={operation}, value={value}, mode={mode}")

sig = signature(configure_system)
partial = sig.bind_partial(operation='initialize')

# Later in the code, when more information is available:
partial.arguments.update(value=10, mode='verbose')
partial.apply_defaults()

configure_system(*partial.args, **partial.kwargs)


Configuring system with operation=initialize, value=10, mode=verbose


In [15]:
def generic_function(x, y, z=100):
    return x + y + z
print('generic_function:',generic_function(1, 2))

def create_context_specific_function(func, **preset_args):
    sig = signature(func)
    bound = sig.bind_partial(**preset_args)

    def wrapper(*args, **kwargs):
        bound_args = bound.arguments.copy()
        bound_args.update(kwargs)
        return func(*args, **bound_args)
    return wrapper

special_func = create_context_specific_function(generic_function, z=500)
signature(special_func)
print('special_func:',special_func(1, 2))

generic_function: 103
special_func: 503


# Complex Workflows and Middleware

complex workflows or middleware, where a request might pass through several layers of processing, these methods can ensure that each layer correctly receives and handles the parameters it needs.

Multi-stage Processing: They facilitate the creation of processing pipelines where each stage can dynamically determine whether it can handle a request based on the parameters it receives.
Practical Example
Imagine a web application framework where controller functions take various parameters: some mandatory, some optional. Before routing a request to a controller, the framework can use partial_bind to attach available query parameters and bind to ensure all required parameters are present before calling the controller, thus avoiding runtime errors and simplifying error handling.

By using bind and partial_bind, developers can create more robust, adaptable, and maintainable code structures that handle a wide range of dynamic calling scenarios efficiently.

In [23]:
def middleware(request, response, func, **route_kwargs):
    sig = signature(func)
    try:
        bound_args = sig.bind(request=request, response=response, **route_kwargs)
        return func(**bound_args.arguments)
    except TypeError as e:
        response.status_code = 400
        return f"Bad request: {e}"

def controller(request, response, user_id):
    return f"user id is {user_id}"

middleware({}, {}, controller, user_id=1234)
        

'user id is 1234'

# functools.partial 

In [24]:
import functools

def power(base, exponent):
    return base ** exponent

# Create a new function that always squares the number
square = functools.partial(power, exponent=2)

print(square(4))  # Outputs: 16
print(square(5))  # Outputs: 25

16

25


In [25]:
def greet(name, greeting):
    print(f"{greeting}, {name}!")

wish_hello = functools.partial(greet, greeting="Hello")
wish_hello("Alice")  # Outputs: Hello, Alice!


Hello, Alice!


In [31]:
from ipywidgets import widgets, Button, Label, Text
import functools
def update_label(label, text):
    label.value = text.value
text_input = Text(value='Hello World')
label = Label()  # Start with an empty label
button = Button(description="Update Label")
# Create a new function that already knows which label to update
update_action = functools.partial(update_label, label)
def on_button_click(b):
    update_action(text=text_input)  # Only need to pass the text input now

button.on_click(on_button_click)
display(text_input, button, label)


Text(value='Hello World')

Button(description='Update Label', style=ButtonStyle())

Label(value='')